### **이미지 분석에서의 통계 분석**
* 구간 척도: 이미지 픽셀 값의 분포를 확인하여 밝기 확인.
    * 픽셀이 많이 분포한 구간을 파악하여 0~255 사이의 값 분포에 따라 밝고 어두움 파악 가능
* 히스토그램: 픽셀 값의 분포를 확인

**분산(Variance):** 데이터 집합 내 값들이 평균으로부터 얼마나 퍼져있는지를 나타내는 측도입니다. 분산은 각 데이터 포인트와 평균 간의 차이를 제곱하고, 그 제곱값들의 평균으로 계산됩니다. 분산은 제곱값을 사용하기 때문에 원래 데이터의 단위와는 다르게 제곱된 단위를 가지게 됩니다.

**편차(Deviation):** 각 데이터 포인트가 평균으로부터 얼마나 떨어져 있는지를 나타내는 값입니다. 편차는 단순히 각 데이터 값에서 평균을 뺀 것입니다. 하지만 편차의 합은 항상 0이므로 편차를 단독으로 사용하면 데이터의 분포를 파악하는 데 한계가 있습니다.

**표준편차(Standard Deviation):** 분산의 양의 제곱근으로, 데이터의 산포 정도를 나타내는 또 다른 측도입니다. 표준편차는 원래 데이터의 단위와 동일한 단위를 가지며, 분산보다 해석이 용이합니다. 표준편차는 분산과 달리 제곱을 하지 않기 때문에 원래 데이터의 단위를 그대로 유지하면서 데이터의 산포 정도를 표현할 수 있습니다.

In [1]:
import numpy as np

def calculate_variance(data):
    mean = np.mean(data)
    variance = np.mean((data - mean) ** 2)
    return variance

def calculate_standard_deviation(data):
    variance = calculate_variance(data)
    std_deviation = np.sqrt(variance)
    return std_deviation

# 예시 데이터
data = np.array([1, 2, 3, 4, 5])

# 분산 계산
variance_result = calculate_variance(data)
print("분산:", variance_result)

# 표준편차 계산
std_deviation_result = calculate_standard_deviation(data)
print("표준편차:", std_deviation_result)


분산: 2.0
표준편차: 1.4142135623730951


In [ ]:
### OpenCV에서 기초 통계량 확인하기
import cv2
import numpy as np

# 이미지 불러오기
image = cv2.imread('your_image_path.jpg')

# 이미지를 1차원 배열로 변환
pixels = image.flatten()

# 평균과 표준편차 계산
mean, std_dev = cv2.meanStdDev(pixels)

print("평균:", mean)
print("표준편차:", std_dev)


In [3]:
import cv2
import mediapipe as mp

# Mediapipe 초기화
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

def detect_eyes(image):
    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.2) as face_mesh:
        # 이미지를 BGR에서 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 미디어파이프에 이미지 전달
        results = face_mesh.process(image_rgb)

        # 결과에서 눈 좌표 추출
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            left_eye = [landmark for landmark in face_landmarks.landmark if 145 <= landmark.x * image.shape[1] <= 175 and 120 <= landmark.y * image.shape[0] <= 150]
            right_eye = [landmark for landmark in face_landmarks.landmark if 260 <= landmark.x * image.shape[1] <= 290 and 120 <= landmark.y * image.shape[0] <= 150]

            return left_eye, right_eye

    return None, None

# 웹캠에서 영상 읽기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 눈 좌표 추적
    left_eye, right_eye = detect_eyes(frame)

    # 결과 표시
    if left_eye and right_eye:
        for eye in [left_eye, right_eye]:
            for landmark in eye:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    # 화면에 출력
    cv2.imshow('Eye Tracking', frame)

    # 종료 조건
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키를 누르면 종료
        break

# 해제
cap.release()
cv2.destroyAllWindows()
